In [ ]:
import openpyxl as oxl
from hpath_backend import excel as xl

In [ ]:
wbook = oxl.load_workbook('../frontend/static/examples/config.xlsx', data_only=True)

In [ ]:
data = xl.get_named_matrix(wbook, index_name='LocationNames', data_name='RunnerTimes')

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data).T
df.insert(0,'Specimen Reception', float('nan'))
df.fillna(0, inplace=True)
runner_times = df + df.T
runner_times

## Data validation and serialisation

In [ ]:
import pydantic as pyd

In [ ]:
class RunnerTimeRow(pyd.BaseModel):
    model_config = pyd.ConfigDict(populate_by_name=True)

    specimen_reception: pyd.NonNegativeFloat = pyd.Field(alias='Specimen Reception')
    lilac_room: pyd.NonNegativeFloat = pyd.Field(alias='Lilac Room')
    white_room: pyd.NonNegativeFloat = pyd.Field(alias='White Room')
    yellow_room: pyd.NonNegativeFloat = pyd.Field(alias='Yellow Room')
    green_room: pyd.NonNegativeFloat = pyd.Field(alias='Green Room')
    processing_room: pyd.NonNegativeFloat = pyd.Field(alias='Processing Room')
    first_floor_corridor_d7: pyd.NonNegativeFloat = pyd.Field(alias='First Floor Corridor D7')
    main_lab: pyd.NonNegativeFloat = pyd.Field(alias='Main Lab')
    staining_room: pyd.NonNegativeFloat = pyd.Field(alias='Staining Room')
    second_floor_landing: pyd.NonNegativeFloat = pyd.Field(alias='Second Floor Landing')
    second_floor_lift_door: pyd.NonNegativeFloat = pyd.Field(alias='Second Floor Lift Door')
    first_floor_landing: pyd.NonNegativeFloat = pyd.Field(alias='First Floor Landing')
    first_floor_lift_door: pyd.NonNegativeFloat = pyd.Field(alias='First Floor Lift Door')
    first_floor_corridor_d14: pyd.NonNegativeFloat = pyd.Field(alias='First Floor Corridor D14')
    first_floor_corridor_d15: pyd.NonNegativeFloat = pyd.Field(alias='First Floor Corridor D15')
    digital_pathology: pyd.NonNegativeFloat = pyd.Field(alias='Digital Pathology')

class RunnerTimes(pyd.BaseModel):
    model_config = pyd.ConfigDict(populate_by_name=True)

    specimen_reception: RunnerTimeRow = pyd.Field(alias='Specimen Reception')
    lilac_room: RunnerTimeRow = pyd.Field(alias='Lilac Room')
    white_room: RunnerTimeRow = pyd.Field(alias='White Room')
    yellow_room: RunnerTimeRow = pyd.Field(alias='Yellow Room')
    green_room: RunnerTimeRow = pyd.Field(alias='Green Room')
    processing_room: RunnerTimeRow = pyd.Field(alias='Processing Room')
    first_floor_corridor_d7: RunnerTimeRow = pyd.Field(alias='First Floor Corridor D7')
    main_lab: RunnerTimeRow = pyd.Field(alias='Main Lab')
    staining_room: RunnerTimeRow = pyd.Field(alias='Staining Room')
    second_floor_landing: RunnerTimeRow = pyd.Field(alias='Second Floor Landing')
    second_floor_lift_door: RunnerTimeRow = pyd.Field(alias='Second Floor Lift Door')
    first_floor_landing: RunnerTimeRow = pyd.Field(alias='First Floor Landing')
    first_floor_lift_door: RunnerTimeRow = pyd.Field(alias='First Floor Lift Door')
    first_floor_corridor_d14: RunnerTimeRow = pyd.Field(alias='First Floor Corridor D14')
    first_floor_corridor_d15: RunnerTimeRow = pyd.Field(alias='First Floor Corridor D15')
    digital_pathology: RunnerTimeRow = pyd.Field(alias='Digital Pathology')

In [ ]:
runner_times_pyd = RunnerTimes(**runner_times.to_dict())

In [ ]:
runner_times_pyd.main_lab.staining_room

In [ ]:
runner_times_pyd.model_dump()

## Network analysis

In [ ]:
import networkx as nx

In [ ]:
df2 = pd.DataFrame(runner_times_pyd.model_dump())

In [ ]:
df2

In [ ]:
g = nx.from_pandas_adjacency(df2)

In [ ]:
list(g.nodes.items())

In [ ]:
list(g.edges.items())

In [ ]:
dict(nx.shortest_path_length(g, weight='weight'))

In [ ]:
runner_times_paths = RunnerTimes(**dict(nx.shortest_path_length(g, weight='weight')))
print(runner_times_paths.staining_room.main_lab)
print(runner_times_paths.main_lab.digital_pathology)

## Final

In [ ]:
df = pd.DataFrame(data).T
df.insert(0,'Specimen Reception', float('nan'))
df.fillna(0, inplace=True)
runner_times = df + df.T
g = nx.from_pandas_adjacency(runner_times)
runner_times_paths = RunnerTimes(**dict(nx.shortest_path_length(g, weight='weight')))

In [ ]:
print(runner_times_paths.staining_room.main_lab)
print(runner_times_paths.main_lab.digital_pathology)

In [ ]:
runner_times_paths.model_dump()